<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/master/Code_of_simple_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import fnmatch
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#Copy images from repo to colab
!git clone https://github.com/SilvesterHsu/ORLFaceRecognition-PCA.git

Cloning into 'ORLFaceRecognition-PCA'...
remote: Enumerating objects: 604, done.
remote: Total 604 (delta 0), reused 0 (delta 0), pack-reused 604
Receiving objects: 100% (604/604), 3.59 MiB | 8.14 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [0]:
# Define block of different types of normalization

In [20]:
dataset = []
#dataset_images = []
indexes = []
test_matrix = []
train_matrix = []
test_labels = []
train_labels = []
dataset_shuffle = False
stratify = None
random_state = 42
path_x=[]
img_list = []
train_images = []
print_difference_photos = False
scaler_enum = ['s0_1','s_1_1','s_std','s_min_max']

# read folder structure with files and put it to collection
folder = []
for i in os.walk(os.path.join('ORLFaceRecognition-PCA','att_faces')):
  folder.append(i)

for address, dirs, files in folder:
  for file in fnmatch.filter(files, '*.pgm'):
    path_x.append(os.path.join(address,file))

for file in sorted(path_x, key=lambda x: int(x.split("/",3)[3].rstrip('.pgm')+x.split("/",3)[2].lstrip("s").zfill(2))):

  img = cv2.imread(file)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
  gray = gray.astype(float)
  person = file.split("/",3)[2].lstrip("s")

  #dataset_images.append(img)
  #dataset.append(gray.flatten().reshape(1, 10304))
  dataset.append(gray.flatten())
  indexes.append(int(person))
  img_list.append(file.split("/",3)[3].rstrip('.pgm'))

# Normalization block
# numbers to range 0 - 1

dataset = np.array(dataset)
#print(dataset.shape)
print(dataset)
dataset_avg = np.average(dataset, axis=0)
#print(dataset_avg)
def f_0_1(a,x_min,x_max,x_avg):
  """Average first and last element of a 1-D array"""
  return a/x_max if a > x_avg else a/x_min if 0 < a < x_avg else 0

vectorized_f_0_1 = np.vectorize(f_0_1)

def f_min_max(a):
  min_a = a.min()
  max_a = a.max()
  avg_a = np.average(a)
  print("Min {}, Max {}",x_min, x_max)
  return vectorized_f_0_1(a,a.min(),a.max(),np.average(a))



#dataset=dataset.swapaxes(0,1)
#print(dataset)

np.apply_along_axis(f_min_max,0,dataset)
print(dataset)
#for i in range(0, len(dataset_avg)):
#  dataset

# std
# -1 : 1

# Without shuffle 1,10
# With shuffle 10,19
# Both 1,19
#for split_mark in range(1, 19):
for split_mark in range(1, 10):

  if split_mark>=10:
    dataset_shuffle = True
    split_k=split_mark-9
  else:
    split_k=split_mark

  train_matrix, test_matrix, train_labels, test_labels = train_test_split(dataset, indexes, test_size=split_k/10, random_state=random_state, shuffle=dataset_shuffle, stratify=stratify)

  estimations_mse = []
  estimations_mae = []
  #estimations_msle = []
  #estimations_std = []

  train_images = []

  for test_image in test_matrix:

    matrix_mse = []
    matrix_mae = []
    #matrix_msle = []
    #matrix_std = []

    for train_image in train_matrix:

      matrix_mse.append(mean_squared_error(test_image, train_image))
      matrix_mae.append(mean_absolute_error(test_image, train_image))
      #matrix_msle.append(mean_squared_log_error(test_image, train_image))
      #matrix_std.append(np.sqrt(mean_squared_error(test_image, train_image)))

    estimations_mse.append(train_labels[np.argmin(matrix_mse)])
    estimations_mae.append(train_labels[np.argmin(matrix_mae)])
    #estimations_msle.append(train_labels[np.argmin(matrix_msle)])
    #estimations_std.append(train_labels[np.argmin(matrix_std)])

    train_images.append(train_matrix[np.argmin(matrix_mse)]) # for print

  result_mse = np.equal(test_labels, estimations_mse)
  result_mae = np.equal(test_labels, estimations_mae)
  #result_msle = np.equal(test_labels, estimations_msle)
  #result_std = np.equal(test_labels, estimations_std)

  matrix_length = len(result_mse)

  evaluation_mse = np.sum(result_mse)/matrix_length
  evaluation_mae = np.sum(result_mae)/matrix_length
  #evaluation_msle = np.sum(result_msle)/matrix_length
  #evaluation_std = np.sum(result_std)/matrix_length

  print('MSE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_mse))
  print('MAE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_mae))
  #print('MSLE{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_msle))
  #print('STD{:s}.  Train/Test : {:d} / {:d} - {:.3f}'.format(' with Shuffle' if dataset_shuffle else '',100-split_k*10, split_k*10, evaluation_std))

  # print dataset distribution
#  if dataset_shuffle:
#    plt.hist(test_labels, bins=range(1, len(list(set(test_labels))) + 2))
#    plt.yticks(np.arange(0, len(list(set(img_list))) + 1, 1))
#    plt.show()
  
  if print_difference_photos:
    for img_ind in range(1,len(result_mse)):
      if result_mse[img_ind]==False:
        img_src = test_matrix[img_ind].reshape(112,92)
        print("Index {}, test:", img_ind)
        print("Index {}, train:", img_ind)
        img_train = train_images[img_ind].reshape(112,92)
        plt.subplot(1,2,1),plt.imshow(img_src)
        plt.title("Test photo")
        plt.subplot(1,2,2),plt.imshow(img_train)
        plt.title("Train photo")
        plt.show()



[[ 48.  49.  45. ...  47.  46.  46.]
 [ 35.  36.  37. ... 144. 147. 143.]
 [103. 105. 104. ...  44.  38.  43.]
 ...
 [108. 105. 104. ...  72.  73.  48.]
 [ 89.  87.  92. ...  93. 112. 109.]
 [125. 124. 124. ...  36.  35.  34.]]


NameError: ignored